# Fantasy Football Team Generation

### Libraries

In [1]:
try: 
    import pandas as pd
    from itertools import combinations
    import ipywidgets as widgets
    from IPython.display import display, clear_output,HTML , Javascript 
except:
    ! pip install pandas openpyxl
    ! pip install ipywidgets

### Funtion that will find the best team

In [12]:


def find_best_teams(data, cutoff_oprk,num_teams):
    qb_df = data[data['Position'] == 'QB'].copy()
    rb_df = data[data['Position'] == 'RB'].copy()
    wr_df = data[data['Position'] == 'WR'].copy()
    te_df = data[data['Position'] == 'TE'].copy()
    dst_df = data[data['Position'] == 'DST'].copy()

    for df in [qb_df, rb_df, wr_df, te_df, dst_df]:
        df['selected'] = False

    flex_df = data[data['Roster Position'].str.contains('FLEX')].copy()
    flex_df['selected'] = False

    for df in [qb_df, rb_df, wr_df, te_df, dst_df, flex_df]:
        df.sort_values(by='AvgPointsPerGame', ascending=False, inplace=True)

    budget = 50000
    #num_teams = 10
    best_teams = []
    message = ""
#     def calculate_team_points(team):
#         weighted_points = 0
#         total_weight = 0

#         for player in team:
#             player_info = data[data['Name'] == player]
#             avg_points = player_info['AvgPointsPerGame'].values[0]
#             oprk = player_info['OPRK'].values[0]

#             # Check for zero division (to avoid divide by zero errors)
#             if avg_points == 0:
#                 weight = 0
#             else:
#                 weight = oprk / avg_points

#             weighted_points += avg_points * weight
#             total_weight += weight

#         return weighted_points / total_weight if total_weight != 0 else 0
    def calculate_team_points(team):
        return sum(data[data['Name'] == player]['AvgPointsPerGame'].values[0] for player in team)

    def calculate_team_salary(team):
        return sum(data[data['Name'] == player]['Salary'].values[0] for player in team)

    def is_valid_team(team):
        # Ensure there is at least one QB, two RBs, three WRs, one TE, one DST, and one FLEX
        positions_count = {
            'QB': 0,
            'RB': 0,
            'WR': 0,
            'TE': 0,
            'DST': 0,
            'FLEX': 0
        }
        for position, players in team.items():
            for player in players:
                positions_count[position] += 1

        if (positions_count['QB'] >= 1 and
            positions_count['RB'] >= 2 and
            positions_count['WR'] >= 3 and  # Change to select 3 WRs
            positions_count['TE'] >= 1 and  # Change to select 1 TE
            positions_count['DST'] >= 1 and
            positions_count['FLEX'] >= 1):
            return True

        return False
    max_iterations = 10000  # Set a maximum number of iterations to prevent an infinite loop
    iterations = 0  # Initialize the iteration counter
    player_short = False
    while len(best_teams) < num_teams:
        iterations +=1
        #print(iterations)
        team = {
            'QB': [],
            'RB': [],
            'WR': [],
            'TE': [],
            'DST': [],
            'FLEX': []
        }

        positions = {
            'QB': (qb_df, 1),
            'RB': (rb_df, 2),
            'WR': (wr_df, 3),  # Changed to 3 WR positions
            'TE': (te_df, 1),
            'DST': (dst_df, 1),
            'FLEX': (flex_df, 1)
        }

        remaining_budget = budget
        for position, (position_df, limit) in positions.items():
            #print(position_df.shape)
            available_players = position_df[~position_df['selected']]
            #Filter players based on OPRK cutoff
            #available_players = available_players[available_players['AvgPointsPerGame'] > 0]
          
            available_players = available_players[available_players['OPRK'] >= cutoff_oprk]
            
            #print(f" Based on criteria Out of {position_df.shape[0]} players : {available_players.shape[0]} Were available")
            #print(f"{len(available_players)} {limit}")
            
            if len(available_players) < limit:
                player_short = True  # Not enough players for this position

            combinations_for_position = combinations(available_players['Name'], limit)
            valid_teams = []

            for combo in combinations_for_position:
                if calculate_team_salary(combo) <= remaining_budget:
                    valid_teams.append(combo)

            if not valid_teams:
                break  # No valid teams for this position

            # Choose the team with the highest total points
            best_team_for_position = max(valid_teams, key=calculate_team_points)
            team[position] = best_team_for_position

            # Mark the selected players as 'selected'
            position_df.loc[position_df['Name'].isin(best_team_for_position), 'selected'] = True

            # Update the remaining budget
            remaining_budget -= calculate_team_salary(best_team_for_position)
        if player_short:
            message = "Number of avilable player of a possition is less then required. Consider adjusting parameters."
            break
        if is_valid_team(team):
            best_teams.append(team)
        if iterations >= max_iterations:
            message = "Maximum number of iterations reached. Consider adjusting parameters."
            break 
    if len(best_teams) == num_teams:
        message = "Found the rquested number of teams"
    # Create a list of DataFrames for each team
    best_teams_df = []
    for i, team in enumerate(best_teams):
        team_data = []
        total_points = 0
        total_salary = 0

        for position, players in team.items():
            for player in players:
                player_info = data[data['Name'] == player][['Roster Position', 'Name', 'Salary', 'AvgPointsPerGame', 'OPRK']]
                team_data.append(player_info.values[0])

                # Calculate total points and total salary for the team
                total_points += player_info['AvgPointsPerGame'].values[0]
                total_salary += player_info['Salary'].values[0]

        # Create a DataFrame for the team
        team_df = pd.DataFrame(team_data, columns=['Roster Position', 'Name', 'Price', 'Avg Points Per Game', 'OPRK'])
        team_df['Team'] = f'Team {i + 1}'
        team_df['Total Points'] = total_points
        team_df['Total Salary'] = total_salary
        best_teams_df.append(team_df)

    return message,best_teams_df


## Run this code to Give Input and View Result

In [19]:
# Create widgets for file upload and cutoff_oprk input
file_upload = widgets.FileUpload(accept='.xlsx', description='Upload Excel file')
cutoff_oprk = widgets.IntText(value=5, description='OPRK Cutoff')
num_teams = widgets.IntText(value=10, description='Num of Team')

# Define output widgets
output_table = widgets.Output()


def on_button_click(b):

    with output_table:
        clear_output()
        
        if file_upload.value:
            
            # Display a "Working..." message
            display(HTML("<h3>Working...</h3>"))
            
            uploaded_file = list(file_upload.value)[0]  # Access the first uploaded file
            file_content = uploaded_file['content']
            with open(uploaded_file['name'], 'wb') as f:
                f.write(file_content)
            
            data = pd.read_excel(uploaded_file['name'])
            cutoff = cutoff_oprk.value
            message, best_teams_df = find_best_teams(data, cutoff,num_teams.value)
            
            # Display each team dataframe
            clear_output()
            print(message)
            for team_df in best_teams_df:
                team_name = team_df['Team'].iloc[0]
                total_points = team_df['Total Points'].iloc[0]
                total_salary = team_df['Total Salary'].iloc[0]
                print(f"Team {team_name}: Total Points = {total_points}, Total Salary = {total_salary}")
                display(team_df[["Roster Position","Name","Price","Avg Points Per Game","OPRK"]])
        else:
            # Display a pop-up message if no file is selected
            display(Javascript("alert('Please select an Excel file before clicking submit.');"))
            return  # Exit the function
        
# Create a button widget to trigger the team calculation
calculate_button = widgets.Button(description='Calculate Best Teams')
calculate_button.on_click(on_button_click)

# Display widgets
display(file_upload, cutoff_oprk,num_teams, calculate_button)
display(output_table)

FileUpload(value=(), accept='.xlsx', description='Upload Excel file')

IntText(value=5, description='OPRK Cutoff')

IntText(value=10, description='Num of Team')

Button(description='Calculate Best Teams', style=ButtonStyle())

Output()